In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import re
import pandas as pd

# selenium에서 사용할 웹 드라이버 절대 경로 정보
chromedriver = 'C:/Users/yeons/2022/Deep daiv NLP RecSys Project/chromedriver_win32/chromedriver'
# selenum의 webdriver에 앞서 설치한 chromedirver 연동
driver = webdriver.Chrome(chromedriver)

C:\Users\yeons\AppData\Local\Temp/ipykernel_24396/345001867.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


https://www.musinsa.com/category/001003?sort=emt_high

In [2]:
# driver로 크롤링할 특정(상의) 페이지로 이동
subcategory = {'구두':'005014',
               '로퍼':'005015',
               '힐/펌프스':'005012',
               '플랫 슈즈':'005017',
               '블로퍼':'005019',
               '샌들슈즈':'005004',
               '슬리퍼':'005018',
               '기타 신발':'005006',
               '모카신/보트 슈즈':'005016',
               '부츠':'005011',
               '신발 용품':'005005'}

total_url = {}
for key, value in subcategory.items():
    # sort=emt_high : 후기 순 정렬
    driver.get('https://www.musinsa.com/category/'+value+'?sort=emt_high')
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    
    # 하위카테고리 내 색상 정보 가져오기(색상명)
    color_list = list(map(lambda x : x.text,soup.find_all('div','tooltip_text')))
    # 각 색상별 코드번호 찾아서 색상명과 연결
    color_dict = {}
    for color in color_list:
        color_dict[color] = soup.find('a',attrs={'href':'#','data-type':'add','alt':color}).get('data-code')
    
    color_url = {}
    for color_key, color_value in color_dict.items():
        #컬러별로 선택한 사이트 내에서 후기 순 정렬
        driver.get('https://www.musinsa.com/category/'+value+'?sort=emt_high&color='+color_value)
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
    
        #각 제품별 URL 링크 가져오기
        url_list = list(map(lambda x : x.get('href'),soup.find_all('a', attrs = {'name' : 'goods_link', 'class' : 'img-block'})))
        #color별로 URL 링크 모으기
        color_url[color_key]=url_list
    
    #하위 카테고리 별로 URL 링크 모으기
    total_url[key] = color_url
    
total_url

{'구두': {'흰색': ['https://store.musinsa.com/app/goods/2080118',
   'https://store.musinsa.com/app/goods/2252484',
   'https://store.musinsa.com/app/goods/2036673',
   'https://store.musinsa.com/app/goods/243513',
   'https://store.musinsa.com/app/goods/2263687',
   'https://store.musinsa.com/app/goods/1147453',
   'https://store.musinsa.com/app/goods/2128149',
   'https://store.musinsa.com/app/goods/1953429',
   'https://store.musinsa.com/app/goods/1011573',
   'https://store.musinsa.com/app/goods/1758418',
   'https://store.musinsa.com/app/goods/2062667',
   'https://store.musinsa.com/app/goods/1502548',
   'https://store.musinsa.com/app/goods/171262',
   'https://store.musinsa.com/app/goods/2322215',
   'https://store.musinsa.com/app/goods/1170302',
   'https://store.musinsa.com/app/goods/850714',
   'https://store.musinsa.com/app/goods/2143831',
   'https://store.musinsa.com/app/goods/1993435',
   'https://store.musinsa.com/app/goods/1732424',
   'https://store.musinsa.com/app/goods/1

In [3]:
import pickle

#모은 URL pickle 파일로 저장해두기
with open('total_url.pickle', 'wb') as f:
    pickle.dump(total_url, f, pickle.HIGHEST_PROTOCOL)


In [4]:
#크롤링 정보 저장할 dataframe
total_info = pd.DataFrame([],columns = ['하위카테고리','색상','상품명','상품고유번호','조회수','누적판매수','좋아요','평점','상품태그정보','가격',
                                        '18세이하','19세이상 23세이하','24세이상 28세이하','29세이상 33세이하',
                                        '34세이상 39세이하','40세이상','남성','여성'])

total_info

,하위카테고리,색상,상품명,상품고유번호,조회수,누적판매수,좋아요,평점,상품태그정보,가격,18세이하,19세이상 23세이하,24세이상 28세이하,29세이상 33세이하,34세이상 39세이하,40세이상,남성,여성


In [5]:
#모아둔 각 상품 url 들어가기
for subcategory,color_url in total_url.items():
    for color, url_list in color_url.items():
            for url in url_list:
                print(url)
                driver.get(url)
                time.sleep(2)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                
                title = soup.find('span', 'product_title','em').text.strip().split('\n')[0]
                view = soup.find('strong',id='pageview_1m').text
                if view == '': # 조회수 정보가 없으면 0
                    view = '0'
                else:
                    view = view.split(' ')[0]
                    
                sales = soup.find('strong',id='sales_1y_qty').text
                if sales == '': # 판매수 정보가 없으면 0
                    sales = '0'
                else:
                    sales = sales.split(' ')[0]
                    
                likes = soup.find('span','prd_like_cnt').text
                likes = re.sub('[^\d]','', likes)
                
                rating = soup.find('span','prd-score__rating')
                if rating: #상품평점이 없으면(리뷰가 없으면) 0점
                    rating = rating.text
                else:
                    rating = '0'
                    
                tag = soup.find('li','article-tag-list list')
                if tag: # tag정보 없으면 빈 리스트 []
                    tag = tag.text.strip().split('\n')
                else:
                    tag = []
                    
                price = soup.find('span','product_article_price','goods_price').text.strip()
                price = re.sub('[^\d]','',price[:-1])
                
                if soup.find_all('span','bar_num') != []:
                    age = list(map(lambda x: x.text[:-1] ,soup.find_all('span','bar_num')))
                    age = list(map(lambda x: int(x)/100 if(x!='') else 0 ,age)) #나이별 정보 없으면 0
                else:
                    age = [0,0,0,0,0,0]
                age_under18 = age[0]
                age_19to23 = age[1]
                age_24to28 = age[2]
                age_29to33 = age[3]
                age_34to39 = age[4]
                age_over40 = age[5]
                
                if soup.find_all('dd','label_info_value') != []: #성별 정보 없으면 0
                    sex = list(map(lambda x : int(x.text[:-1])/100 if x.text[:-1] != 'NaN' else 0,soup.find_all('dd','label_info_value')))
                else:
                    sex = [0,0]
                men = sex[0]
                women = sex[1]
                
                info = {'하위카테고리':subcategory,'색상':color,'상품명':title,'상품고유번호':int(url.split('/')[-1]),
                        '조회수':view,'누적판매수':sales,'좋아요':likes,'평점':rating,'상품태그정보':tag,'가격':price,
                        '18세이하':age_under18,'19세이상 23세이하':age_19to23,'24세이상 28세이하':age_24to28,
                        '29세이상 33세이하':age_29to33,'34세이상 39세이하':age_34to39,'40세이상':age_over40,'남성':men,'여성':women}
                total_info = total_info.append(info,ignore_index=True)    
            # 중간에 한번씩 저장해주기
            with open('무신사_total_info.pickle', 'wb') as f:
                pickle.dump(total_info, f, pickle.HIGHEST_PROTOCOL)  
            print('중간 저장')          

total_info

https://store.musinsa.com/app/goods/2080118
https://store.musinsa.com/app/goods/2252484
https://store.musinsa.com/app/goods/2036673
https://store.musinsa.com/app/goods/243513
https://store.musinsa.com/app/goods/2263687
https://store.musinsa.com/app/goods/1147453
https://store.musinsa.com/app/goods/2128149
https://store.musinsa.com/app/goods/1953429
https://store.musinsa.com/app/goods/1011573
https://store.musinsa.com/app/goods/1758418
https://store.musinsa.com/app/goods/2062667
https://store.musinsa.com/app/goods/1502548
https://store.musinsa.com/app/goods/171262
https://store.musinsa.com/app/goods/2322215
https://store.musinsa.com/app/goods/1170302
https://store.musinsa.com/app/goods/850714
https://store.musinsa.com/app/goods/2143831
https://store.musinsa.com/app/goods/1993435
https://store.musinsa.com/app/goods/1732424
https://store.musinsa.com/app/goods/1579241
https://store.musinsa.com/app/goods/1170251
https://store.musinsa.com/app/goods/2420372
https://store.musinsa.com/app/goods

https://store.musinsa.com/app/goods/1563577
https://store.musinsa.com/app/goods/2042398
https://store.musinsa.com/app/goods/804481
https://store.musinsa.com/app/goods/853225
https://store.musinsa.com/app/goods/1109393
https://store.musinsa.com/app/goods/1912672
https://store.musinsa.com/app/goods/1246797
https://store.musinsa.com/app/goods/1273500
https://store.musinsa.com/app/goods/1021395
https://store.musinsa.com/app/goods/2148099
https://store.musinsa.com/app/goods/2143835
https://store.musinsa.com/app/goods/1021422
https://store.musinsa.com/app/goods/1480999
https://store.musinsa.com/app/goods/1127911
중간 저장
https://store.musinsa.com/app/goods/1837347
https://store.musinsa.com/app/goods/1836261
https://store.musinsa.com/app/goods/1598191
https://store.musinsa.com/app/goods/1596814
https://store.musinsa.com/app/goods/1579209
https://store.musinsa.com/app/goods/1564555
https://store.musinsa.com/app/goods/1561290
https://store.musinsa.com/app/goods/1561214
https://store.musinsa.com/ap

https://store.musinsa.com/app/goods/731235
https://store.musinsa.com/app/goods/396045
https://store.musinsa.com/app/goods/1213700
https://store.musinsa.com/app/goods/2138044
https://store.musinsa.com/app/goods/2255042
https://store.musinsa.com/app/goods/2255037
https://store.musinsa.com/app/goods/2207764
https://store.musinsa.com/app/goods/2125161
https://store.musinsa.com/app/goods/2046145
https://store.musinsa.com/app/goods/2032647
https://store.musinsa.com/app/goods/1918419
https://store.musinsa.com/app/goods/1870269
https://store.musinsa.com/app/goods/1819692
https://store.musinsa.com/app/goods/1819385
https://store.musinsa.com/app/goods/1793851
https://store.musinsa.com/app/goods/1776899
https://store.musinsa.com/app/goods/1691872
https://store.musinsa.com/app/goods/1469469
https://store.musinsa.com/app/goods/1453073
https://store.musinsa.com/app/goods/1452278
https://store.musinsa.com/app/goods/1451579
https://store.musinsa.com/app/goods/1233897
https://store.musinsa.com/app/good

https://store.musinsa.com/app/goods/1423082
https://store.musinsa.com/app/goods/1423084
https://store.musinsa.com/app/goods/2083515
https://store.musinsa.com/app/goods/1423086
https://store.musinsa.com/app/goods/2235469
https://store.musinsa.com/app/goods/1745785
https://store.musinsa.com/app/goods/1745778
https://store.musinsa.com/app/goods/2218635
https://store.musinsa.com/app/goods/1745782
https://store.musinsa.com/app/goods/1745783
https://store.musinsa.com/app/goods/1768795
https://store.musinsa.com/app/goods/1745791
https://store.musinsa.com/app/goods/1423089
https://store.musinsa.com/app/goods/2274011
https://store.musinsa.com/app/goods/1423077
https://store.musinsa.com/app/goods/1768793
https://store.musinsa.com/app/goods/1761741
https://store.musinsa.com/app/goods/1745793
https://store.musinsa.com/app/goods/1768794
https://store.musinsa.com/app/goods/1768792
https://store.musinsa.com/app/goods/1423075
https://store.musinsa.com/app/goods/2083519
https://store.musinsa.com/app/go

https://store.musinsa.com/app/goods/1568266
https://store.musinsa.com/app/goods/1562245
https://store.musinsa.com/app/goods/1452349
중간 저장
https://store.musinsa.com/app/goods/1618424
https://store.musinsa.com/app/goods/1815667
https://store.musinsa.com/app/goods/424885
https://store.musinsa.com/app/goods/1981974
https://store.musinsa.com/app/goods/1177469
https://store.musinsa.com/app/goods/2204874
https://store.musinsa.com/app/goods/2154064
https://store.musinsa.com/app/goods/1882474
https://store.musinsa.com/app/goods/1574423
https://store.musinsa.com/app/goods/299333
https://store.musinsa.com/app/goods/2413096
https://store.musinsa.com/app/goods/2301282
https://store.musinsa.com/app/goods/2230610
https://store.musinsa.com/app/goods/1992731
https://store.musinsa.com/app/goods/1974916
https://store.musinsa.com/app/goods/1925359
https://store.musinsa.com/app/goods/1869933
https://store.musinsa.com/app/goods/1787973
https://store.musinsa.com/app/goods/1732395
https://store.musinsa.com/ap

...

https://store.musinsa.com/app/goods/1154010
https://store.musinsa.com/app/goods/1124364
https://store.musinsa.com/app/goods/1108477
https://store.musinsa.com/app/goods/1060234
https://store.musinsa.com/app/goods/1011162
https://store.musinsa.com/app/goods/906883
https://store.musinsa.com/app/goods/894124
https://store.musinsa.com/app/goods/867425
https://store.musinsa.com/app/goods/779115
https://store.musinsa.com/app/goods/774610
https://store.musinsa.com/app/goods/648743
https://store.musinsa.com/app/goods/576685
https://store.musinsa.com/app/goods/576612
https://store.musinsa.com/app/goods/540383
https://store.musinsa.com/app/goods/414000
https://store.musinsa.com/app/goods/413823
https://store.musinsa.com/app/goods/413817
https://store.musinsa.com/app/goods/381220
중간 저장
https://store.musinsa.com/app/goods/2004186
중간 저장
https://store.musinsa.com/app/goods/2377311
https://store.musinsa.com/app/goods/2374824
https://store.musinsa.com/app/goods/1964089
중간 저장
https://store.musinsa.com/a

https://store.musinsa.com/app/goods/1503515
중간 저장
https://store.musinsa.com/app/goods/1989912
https://store.musinsa.com/app/goods/1827790
https://store.musinsa.com/app/goods/1705726
중간 저장
https://store.musinsa.com/app/goods/1992362
https://store.musinsa.com/app/goods/1966982
https://store.musinsa.com/app/goods/1503474
중간 저장
https://store.musinsa.com/app/goods/1888860
중간 저장
https://store.musinsa.com/app/goods/2170188
https://store.musinsa.com/app/goods/2170183
https://store.musinsa.com/app/goods/2170174
https://store.musinsa.com/app/goods/1975126
https://store.musinsa.com/app/goods/1952567
https://store.musinsa.com/app/goods/1837632
https://store.musinsa.com/app/goods/1791756
https://store.musinsa.com/app/goods/1714401
https://store.musinsa.com/app/goods/1651394
https://store.musinsa.com/app/goods/1502989
https://store.musinsa.com/app/goods/1502980
https://store.musinsa.com/app/goods/1227859
https://store.musinsa.com/app/goods/898080
https://store.musinsa.com/app/goods/894045
https://st

https://store.musinsa.com/app/goods/1968681
https://store.musinsa.com/app/goods/1966065
https://store.musinsa.com/app/goods/1966028
https://store.musinsa.com/app/goods/1966027
https://store.musinsa.com/app/goods/1963359
https://store.musinsa.com/app/goods/1962978
https://store.musinsa.com/app/goods/1890499
https://store.musinsa.com/app/goods/1860114
https://store.musinsa.com/app/goods/1853857
https://store.musinsa.com/app/goods/1851060
https://store.musinsa.com/app/goods/1840990
https://store.musinsa.com/app/goods/1839324
https://store.musinsa.com/app/goods/1798894
https://store.musinsa.com/app/goods/1776594
https://store.musinsa.com/app/goods/1776507
https://store.musinsa.com/app/goods/1759070
https://store.musinsa.com/app/goods/1721744
https://store.musinsa.com/app/goods/1624850
https://store.musinsa.com/app/goods/1622187
https://store.musinsa.com/app/goods/1622173
https://store.musinsa.com/app/goods/1503449
https://store.musinsa.com/app/goods/1428702
https://store.musinsa.com/app/go

https://store.musinsa.com/app/goods/2397802
https://store.musinsa.com/app/goods/2386507
https://store.musinsa.com/app/goods/2322774
https://store.musinsa.com/app/goods/2194262
https://store.musinsa.com/app/goods/2101147
https://store.musinsa.com/app/goods/2012378
https://store.musinsa.com/app/goods/1969269
https://store.musinsa.com/app/goods/1969266
https://store.musinsa.com/app/goods/1827814
https://store.musinsa.com/app/goods/1827791
https://store.musinsa.com/app/goods/1810715
https://store.musinsa.com/app/goods/1809801
https://store.musinsa.com/app/goods/1809775
https://store.musinsa.com/app/goods/1809689
https://store.musinsa.com/app/goods/1760252
https://store.musinsa.com/app/goods/1740975
https://store.musinsa.com/app/goods/1697900
https://store.musinsa.com/app/goods/1659028
https://store.musinsa.com/app/goods/1632412
https://store.musinsa.com/app/goods/1597959
https://store.musinsa.com/app/goods/1570679
https://store.musinsa.com/app/goods/1351594
https://store.musinsa.com/app/go

ValueError: invalid literal for int() with base 10: 'NaN'

In [6]:
#추후에 pickle 불러와서 csv로 저장하기
with open('무신사_total_info.pickle', 'rb') as f:
    data = pickle.load(f)

data.to_csv('무신사 크롤링정보_신발.csv')

In [2]:
import pandas as pd

data = pd.read_csv('무신사 크롤링정보_신발.csv')
data.head()

,Unnamed: 0,하위카테고리,색상,상품명,상품고유번호,조회수,누적판매수,좋아요,평점,상품태그정보,가격,18세이하,19세이상 23세이하,24세이상 28세이하,29세이상 33세이하,34세이상 39세이하,40세이상,남성,여성
0,0,구두,흰색,스퀘어 마틴 더비 슈즈 aaa300m(WHITE),2080118,1만,200개,1528,4.7,"['#남자더비슈즈', '#더비슈즈', '#구두']",298000,0.14,0.25,0.38,0.16,0.04,0.03,0.90,0.10
1,1,구두,흰색,3/18 배송,2252484,7.1천,200개,908,4.7,"['#더비슈즈', '#남자구두', '#여자구두', '#스퀘어']",79000,0.12,0.37,0.29,0.13,0.04,0.05,0.85,0.15
2,2,구두,흰색,1461 페이턴트 화이트 / 26754100,2036673,1.7천,50개,481,4.8,"['#화이트', '#베이식', '#앵클부츠', '#14613홀', '#더비슈즈', ...",190000,0.27,0.27,0.22,0.10,0.06,0.08,0.57,0.43
3,3,구두,흰색,V6803 블랙 인터레이스 화이트 레더 비바 몬도 솔 크리퍼,243513,0,0,199,5.0,['#블랙'],139800,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,4,구두,흰색,Forik_wt,2263687,2.7천,50개,303,4.9,[],285000,0.10,0.24,0.37,0.17,0.08,0.04,0.93,0.07
